In [1]:
%load_ext autoreload
%autoreload 2

from datasets.kaggle import KaggleDataset
import pandas as pd
import numpy as np
import plotly.express as px

np.set_printoptions(precision=4, suppress=True)

RAND_GEN = np.random.default_rng(239873)
REPETITIONS = 10
WEIGHT_RANGE = (.01, 1)
ALPHA_RANGE = (.01, 1)

In [2]:
dataset = KaggleDataset("Daily")

train_price, _ = dataset.get_price_date_series(
    pd.Timestamp("2014-1-1"), pd.Timestamp("2020-1-1")
)
test_price, _ = dataset.get_price_date_series(
    pd.Timestamp("2020-1-1"), pd.Timestamp("2030-1-1")
)

In [3]:
from algos.hill import HC
from algos.pso import PSO
from algos.woa import WOA

algo = {"hill": HC, "pso": PSO, "woa": WOA}

In [4]:
from bot import sim_bot


def make_eval_fn(signal_fn, data):
    return lambda params: sim_bot(signal_fn(data, *params), data)

In [5]:
from algos.base import BaseAlgo
from signals import complex_bot_signal

complex_eval_fn = make_eval_fn(complex_bot_signal, train_price)


def make_algo_from_HL(algo, h, l):
    return algo(
        complex_eval_fn,
        bounds=[WEIGHT_RANGE] * 3
        + [h] * 3
        + [ALPHA_RANGE]
        + [WEIGHT_RANGE] * 3
        + [l] * 3
        + [ALPHA_RANGE],
        integer_dims=[3, 4, 5, 10, 11, 12],
        seed=RAND_GEN,
    )


groupA: dict[str, BaseAlgo] = {
    k: make_algo_from_HL(v, (1, 10), (11, 40)) for k, v in algo.items()
}
groupB: dict[str, BaseAlgo] = {
    k: make_algo_from_HL(v, (5, 50), (51, 100)) for k, v in algo.items()
}


In [6]:
# VIS 1

for algo_name, algo_instance in groupB.items():
    algo_instance.optimise()
    print(f"Algo: {algo_name}\nBest Fitness: {algo_instance.best_fitness}\nBest Params: {algo_instance.best_params}")
    algo_instance.plot().show()

Algo: hill
Best Fitness: 13826.250082403589
Best Params: [0.7228412442139138, 0.7662516329273212, 0.3395601217435938, 23, 9, 35, 0.6679910099903681, 0.7733514278797653, 0.499469654539877, 0.38650501171512597, 51, 79, 78, 0.2701953773838233]


Algo: pso
Best Fitness: 33160.75819564563
Best Params: [  1.       0.1126   0.01    50.       5.      41.       1.       1.
   1.       0.4477 100.      90.      99.       0.101 ]


Algo: woa
Best Fitness: 35564.02634149694
Best Params: [  0.034    0.669    0.0962   8.      40.      25.       0.4388   0.035
   0.01     0.8757  56.      80.     100.       0.01  ]


In [7]:
# VIS 2

for algo_name, algo_instance in groupB.items():
    for _ in range(REPETITIONS - 1):
        algo_instance.optimise()
for algo_name, algo_instance in groupA.items():
    for _ in range(REPETITIONS):
        algo_instance.optimise()

def eval_mean(algo, bot_signal: np.ndarray, price: np.ndarray):
    assert isinstance(algo, BaseAlgo)
    return np.mean(algo.historical_best_params_fitness(make_eval_fn(bot_signal, price)))

## TEST

# build a dataframe with the algo name, its best score, and its group
group_A_df_test = pd.DataFrame(
    {
        "algo": [algo_name for algo_name in groupA.keys()],
        "score": [eval_mean(algo, complex_bot_signal, test_price) for algo in groupA.values()],
        "group": ["A"] * len(groupA),
        "type": ["complex"] * len(groupA),
    }
)
group_B_df_test = pd.DataFrame(
    {
        "algo": [algo_name for algo_name in groupB.keys()],
        "score": [eval_mean(algo, complex_bot_signal, test_price) for algo in groupB.values()],
        "group": ["B"] * len(groupB),
        "type": ["complex"] * len(groupB),
    }
)
# combine the two dataframes
combined_df = pd.concat([group_A_df_test, group_B_df_test], ignore_index=True)

px.bar(
    combined_df,
    x="algo",
    y="score",
    color="group",
    barmode="group",
    labels={
        "score": "Optimised Bot's Final Cash ($)",
        "algo": "Optimisation Algorithm",
        "group": "Window Bounds",
    },
    color_discrete_sequence=px.colors.qualitative.Plotly,
).update_traces(texttemplate="%{y:.2f}", textposition="outside").show()

## TRAIN

# build a dataframe with the algo name, its best score, and its group for TRAIN
group_A_df_train = pd.DataFrame(
    {
        "algo": [algo_name for algo_name in groupA.keys()],
        "score": [eval_mean(algo, complex_bot_signal, train_price) for algo in groupA.values()],
        "group": ["A"] * len(groupA),
        "type": ["complex"] * len(groupA),
    }
)
group_B_df_train = pd.DataFrame(
    {
        "algo": [algo_name for algo_name in groupB.keys()],
        "score": [eval_mean(algo, complex_bot_signal, train_price) for algo in groupB.values()],
        "group": ["B"] * len(groupB),
        "type": ["complex"] * len(groupB),
    }
)
# combine the two dataframes
combined_df_train = pd.concat([group_A_df_train, group_B_df_train], ignore_index=True)

px.bar(
    combined_df_train,
    x="algo",
    y="score",
    color="group",
    barmode="group",
    labels={
        "score": "Optimised Bot's Final Cash ($)",
        "algo": "Optimisation Algorithm",
        "group": "Window Bounds",
    },
    color_discrete_sequence=px.colors.qualitative.Plotly,
).update_traces(texttemplate="%{y:.2f}", textposition="outside").show()

In [8]:
# VIS 3

from signals import sma2_bot_signal, smaema_bot_signal


def make_sma2_algo_from_HL(algo, h, l):
    return algo(
        make_eval_fn(sma2_bot_signal, train_price), bounds=[h, l], seed=RAND_GEN
    )


def make_smaema_algo_from_HL(algo, h, l):
    return algo(
        make_eval_fn(smaema_bot_signal, train_price),
        bounds=[h, l, (0.01, 1)],
        seed=RAND_GEN,
    )


groupB_sma2: dict[str, BaseAlgo] = {
    k: make_sma2_algo_from_HL(v, (5, 50), (51, 100)) for k, v in algo.items()
}
groupB_smaema: dict[str, BaseAlgo] = {
    k: make_smaema_algo_from_HL(v, (5, 50), (51, 100)) for k, v in algo.items()
}

# optimise group A and B for sma2 and smaema
for _ in range(REPETITIONS):
    for algo_name, algo_instance in groupB_sma2.items():
        algo_instance.optimise()
    for algo_name, algo_instance in groupB_smaema.items():
        algo_instance.optimise()

# TEST

# build a dataframe that contains algo name, best fitness, group, and type (sma2, smaema, complex)
group_B_sma2_df = pd.DataFrame(
    {
        "algo": [algo_name for algo_name in groupB_sma2.keys()],
        "score": [eval_mean(algo, sma2_bot_signal, test_price) for algo in groupB_sma2.values()],
        "group": ["B"] * len(groupB_sma2),
        "type": ["sma2"] * len(groupB_sma2),
    }
)
group_B_smaema_df = pd.DataFrame(
    {
        "algo": [algo_name for algo_name in groupB_smaema.keys()],
        "score": [eval_mean(algo, smaema_bot_signal, test_price) for algo in groupB_smaema.values()],
        "group": ["B"] * len(groupB_smaema),
        "type": ["smaema"] * len(groupB_smaema),
    }
)

# combine the dataframes
bigger_combined_df = pd.concat(
    [
        group_B_sma2_df,
        group_B_smaema_df,
        group_B_df_test,
    ],
    ignore_index=True,
)

fig = px.bar(
    bigger_combined_df,
    x="algo",
    y="score",
    color="type",
    barmode="group",
    labels={
        "score": "Optimised Bot's Final Cash ($)",
        "algo": "Optimisation Algorithm",
        "type": "Bot Signal Type",
    },
    color_discrete_sequence=px.colors.qualitative.Plotly,
)
fig.update_traces(texttemplate="%{y:.2f}", textposition="outside")
fig.show()

## TRAIN

# build a dataframe that contains algo name, best fitness, group, and type (sma2, smaema, complex)
group_B_sma2_df = pd.DataFrame(
    {
        "algo": [algo_name for algo_name in groupB_sma2.keys()],
        "score": [eval_mean(algo, sma2_bot_signal, train_price) for algo in groupB_sma2.values()],
        "group": ["B"] * len(groupB_sma2),
        "type": ["sma2"] * len(groupB_sma2),
    }
)
group_B_smaema_df = pd.DataFrame(
    {
        "algo": [algo_name for algo_name in groupB_smaema.keys()],
        "score": [eval_mean(algo, smaema_bot_signal, train_price) for algo in groupB_smaema.values()],
        "group": ["B"] * len(groupB_smaema),
        "type": ["smaema"] * len(groupB_smaema),
    }
)

# combine the dataframes
bigger_combined_df = pd.concat(
    [
        group_B_sma2_df,
        group_B_smaema_df,
        group_B_df_train,
    ],
    ignore_index=True,
)

fig = px.bar(
    bigger_combined_df,
    x="algo",
    y="score",
    color="type",
    barmode="group",
    labels={
        "score": "Optimised Bot's Final Cash ($)",
        "algo": "Optimisation Algorithm",
        "type": "Bot Signal Type",
    },
    color_discrete_sequence=px.colors.qualitative.Plotly,
)
fig.update_traces(texttemplate="%{y:.2f}", textposition="outside")
fig.show()


In [9]:
# VIS 4 - show final values over 10 repeated runs

## TEST

repeated = {"algo": [], "score": [], "run": []}

for algo_name, algo_instance in groupB.items():
    for i, fitness in enumerate(algo_instance.historical_best_params_fitness(make_eval_fn(complex_bot_signal, test_price))):
        repeated["algo"].append(algo_name)
        repeated["score"].append(fitness)
        repeated["run"].append(i + 1)

px.line(
    pd.DataFrame(repeated),
    x="run",
    y="score",
    color="algo",
    labels={
        "score": "Optimised Bot's Final Cash ($)",
        "run": "Run Number",
        "algo": "Optimisation Algorithm",
    },
    markers=True,
).update_xaxes(dtick=1).show()

## TRAIN

repeated = {"algo": [], "score": [], "run": []}

for algo_name, algo_instance in groupB.items():
    for i, fitness in enumerate(algo_instance.historical_best_params_fitness(make_eval_fn(complex_bot_signal, train_price))):
        repeated["algo"].append(algo_name)
        repeated["score"].append(fitness)
        repeated["run"].append(i + 1)

px.line(
    pd.DataFrame(repeated),
    x="run",
    y="score",
    color="algo",
    labels={
        "score": "Optimised Bot's Final Cash ($)",
        "run": "Run Number",
        "algo": "Optimisation Algorithm",
    },
    markers=True,
).update_xaxes(dtick=1).show()
